# Load API client

In [ ]:
import os
from mpcontribs.client import load_client

In [ ]:
headers = { # just for local testing, kong+apikey will take care of headers
    'X-Consumer-Groups': 'my_project',
    'X-Consumer-Username': 'patrick@the-huck.com',
    #'X-Consumer-Username': os.environ['USER']
}
client = load_client(headers=headers)
project = 'my_project'

In [ ]:
dir(client)

# Retrieve and update project

In [ ]:
dir(client.projects)

In [ ]:
client.projects.get_entry(pk=project, _fields=['_all']).response().result

In [ ]:
client.projects.update_entry(pk=project, project={
    'title': 'Cool Title', 'other': {'hello': 'world', 'a': {'b': '3 %', 'c': '3.44 meV'}},
    'urls': {'PRC': 'https://google.com', 'PRCa': 'https://google.com', 'PRCb': 'https://google.com'}
}).response().result

# Create, retrieve, and update contributions

In [ ]:
dir(client.contributions)

In [ ]:
identifier = 'mp-1002'
client.contributions.create_entry(contribution={
    'project': project, 'identifier': identifier,
    'data': {'formula': 'CO2', 'E': '3.33 eV', 'E|V': {'a': 1, 'b': '3 cm'}}
}).response().result

In [ ]:
client.contributions.get_entries(project=project, _fields=['_all']).response().result

In [ ]:
client.contributions.update_entry(
    pk='5dfb17e2715f82731a341e8c', contribution={'data': {'formula': 'MnO2'}}
).response().result
# TODO overwrites data!

# Tables

# Structures

In [ ]:
client.contributions.delete_entry(pk='5dfb17e2715f82731a341e8c').response().result

# Add overview graph

In [ ]:
import plotly.express as px

In [ ]:
contribs = client.contributions.get_entries(
    project='MnO2_phase_selection',
    _fields=['id', 'identifier', 'data.ΔH.value']
).response().result['data']

# Publish and/or delete project

In [ ]:
client.projects.delete_entry(pk='my_project').response().result

In [ ]:
# use update_entries to also publish according contributions, tables, and structures